Task 1: Download the Question-and-Answer Dataset  

you can create dataframe to store your jsonal file data

In [ ]:
import pandas as pd
import jsonlines  # Install this library: pip install jsonlines

jsonl_file_path = 'dev.jsonl'

# Initialize an empty list to store data
data_list = []

# Open the JSONL file and read line by line
with jsonlines.open(jsonl_file_path, 'r') as reader:
    for item in reader:
        # Append the record to the list
        data_list.append(item)

# Create DataFrame
df = pd.DataFrame(data_list)

# Display DataFrame info (columns, data types, etc.)
print(df.info())

# Print the number of columns
print(f"Number of columns: {len(df.columns)}")


Above method can display memory issues

You can use below method to discover what kind of dataset you have

In [12]:
# Read only the first 100 rows
import pandas as pd
data_url = "dev.jsonl"
df1 = pd.read_json(data_url, lines=True, nrows=1000)

# Display the data
print(df1)

                                           annotations  \
0    [{'annotation_id': 13591449469826568799, 'long...   
1    [{'annotation_id': 9256543824648308180, 'long_...   
2    [{'annotation_id': 647276088892962831, 'long_a...   
3    [{'annotation_id': 2430977867500315580, 'long_...   
4    [{'annotation_id': 9697565976974065308, 'long_...   
..                                                 ...   
995  [{'annotation_id': 9850947005047531828, 'long_...   
996  [{'annotation_id': 14443401039595128465, 'long...   
997  [{'annotation_id': 13936736338887547733, 'long...   
998  [{'annotation_id': 6001080811847477710, 'long_...   
999  [{'annotation_id': 3767820468905660496, 'long_...   

                                         document_html  \
0    <!DOCTYPE html>\n<HTML class="client-js ve-not...   
1    <!DOCTYPE html>\n<HTML class="client-js ve-not...   
2    <!DOCTYPE html>\n<HTML class="client-js ve-not...   
3    <!DOCTYPE html>\n<HTML class="client-js ve-not...   
4    <!DOCTYP

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   annotations             1000 non-null   object
 1   document_html           1000 non-null   object
 2   document_title          1000 non-null   object
 3   document_tokens         1000 non-null   object
 4   document_url            1000 non-null   object
 5   example_id              1000 non-null   int64 
 6   long_answer_candidates  1000 non-null   object
 7   question_text           1000 non-null   object
 8   question_tokens         1000 non-null   object
dtypes: int64(1), object(8)
memory usage: 70.4+ KB


In [14]:
# After reading the data (your `df1` variable)
print(f"Number of columns: {len(df1.columns)}")
print(f"Column names: {list(df1.columns)}")


Number of columns: 9
Column names: ['annotations', 'document_html', 'document_title', 'document_tokens', 'document_url', 'example_id', 'long_answer_candidates', 'question_text', 'question_tokens']


task 2: Clean the Dataset and Prepare a CSV file

In [ ]:
import pandas as pd
import jsonlines  # Install this library: pip install jsonlines. this is very useful to read large datasets.

jsonl_file_path = 'dev.jsonl'
csv_output_path = 'dev1.csv'

# Initialize lists to store data
document_title_list = []
question_text_list = []
short_answer_list = []
yes_no_answer_list = []


# Open the JSONL file and read line by line
with jsonlines.open(jsonl_file_path, 'r') as reader:
    for item in reader:
        document_title_list.append(item.get('document_title', ''))
        question_text_list.append(item.get('question_text', ''))
        short_answer_list.append(item.get('short_answer',''))
        yes_no_answer_list.append(item.get('yes_no_answer',''))
        
        # Handle multiple short answers
        short_answers = item.get('short_answer', [])
        for i, answer in enumerate(short_answers, start=1):
            short_answer_list.append(f'short_answer{i}' if i > 1 else 'short_answer')
            

# Create DataFrame
df = pd.DataFrame({
    'document_title': document_title_list,
    'question_text': question_text_list,
    'short_answer': short_answer_list,
    'yes_no_answer': yes_no_answer_list,
    
})

# Deal with missing values (example)
df = df.dropna()  # Remove rows with missing values

# Save to CSV
df.to_csv(csv_output_path, index=False)

# Display DataFrame info (columns, data types, etc.)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7830 entries, 0 to 7829
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   document_title  7830 non-null   object
 1   question_text   7830 non-null   object
 2   short_answer    7830 non-null   object
 3   yes_no_answer   7830 non-null   object
dtypes: object(4)
memory usage: 244.8+ KB
None


Task 3: Build a Streamlit web app



let's create a Streamlit app using the cleaned dataset

In [ ]:
import streamlit as st

# Load the CSV
df = pd.read_csv("dev.csv")

# Create filter options
title_filter = st.selectbox("Filter by document_title", df["document_title"].unique())
question_text_filter = st.text_input("Filter by question_text")

# Apply filters
filtered_df = df.loc[(df["document_title"] == title_filter) & (df["question_text"].str.contains(question_text_filter))]

# Streamlit app
st.title('Q&A Session')

# Display the filtered dataset
st.dataframe(filtered_df)
